# Argo / GO-SHIP Comparison

This will serve as our working notebook, broken into our discussed working components below. 

## This cell for imports, constants

In [70]:
# data processing
import numpy as np
import pandas as pd
import xarray as xr
from time import sleep

#data visualization
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline

# API convenience functions
from utilities_NSF_EC2022 import get_data_for_timeRange

import warnings
warnings.filterwarnings('ignore')

# set constants
URL_PREFIX = 'https://argovis-api.colorado.edu'
API_KEY = ''

## This cell for data loading/processing to xarray

In [2]:
# more convenient wrapper for if you want a line just by name
def get_goship_line(line_name, startDate='1900-01-01T00:00:00Z', endDate='2022-05-01T00:00:00Z', dt_tag='365d', url=URL_PREFIX, api_key=API_KEY):
    df = get_data_for_timeRange(startDate, endDate, url_prefix=url+'/profiles?', 
                                source='cchdo_go-ship', woceline=line_name, 
                                myAPIkey=api_key, dt_tag=dt_tag)
    return df

# I think we should have a time independent version of this function as well
# could cheat this function into doing it with something like timedelta=1e5 or something
def get_argo_along_line(time, coords, radius=50, timedelta=30, dt_tag='365d', url=URL_PREFIX, api_key=API_KEY):
    df_all = pd.DataFrame()
    for t, c in zip(time, coords):
        sleep(.2)
        startDate = (pd.Timestamp(t) - pd.Timedelta(timedelta/2)).strftime('%Y-%m-%dT%H:%M:%SZ')
        endDate   = (pd.Timestamp(t) + pd.Timedelta(timedelta/2)).strftime('%Y-%m-%dT%H:%M:%SZ')
        center    = f'{c[0]},{c[1]}'
        df = get_data_for_timeRange(startDate, endDate, url_prefix=url+'/profiles?',
            center=center, radius_km=f'{radius}', source='argo_core', data='pres,temp,psal',
            myAPIkey=api_key, dt_tag=dt_tag, writeFlag=False)
        df_all = df_all.append(df)
    
    return df_all

# get argo data along line A22
a22 = get_goship_line('A22')
coords = [c['coordinates'] for c in a22.geolocation]
time = a22.timestamp.values
argo_a22 = get_argo_along_line(time, coords)

In [74]:
# process data from full of JSON points to more usable form
def json_dataframe_to_dataframe(df):
    out = pd.DataFrame()
    for i in range(df.shape[0]):
        # get the argo data
        data_dict = dict()
        data = df.data.iloc[i]
        # repeat location and time data for same lenth as array
        N_levels = len(data)
        data_dict['wmo'] = N_levels*[int(df._id.iloc[i].split('_')[0])]
        data_dict['cycle_number'] = N_levels*[df.cycle_number.iloc[i]]
        data_dict['time'] = N_levels*[df.timestamp.iloc[i]]
        data_dict['longitude'] = N_levels*[df.geolocation.iloc[i]['coordinates'][0]]
        data_dict['latitude'] = N_levels*[df.geolocation.iloc[i]['coordinates'][1]]
        # extract data from JSON dict
        for k in df.data_keys.iloc[i]:
            data_dict[k] = [d[k] for d in data]

        out = pd.concat((out, pd.DataFrame(data_dict, index=(i+1)*np.ones((N_levels,), dtype=int))))
    
    return out

df = json_dataframe_to_dataframe(argo_a22)

def dataframe_to_xarray(df):
    # get max number of depth levels
    ix = df.index.unique()
    number_profiles = ix.shape[0]
    max_levels = -np.inf
    for nprof in ix:
        if df.loc[nprof].shape[0] > max_levels:
            max_levels = df.loc[nprof].shape[0]

    # extract lower dimension metadata
    metadata = ['wmo', 'cycle_number', 'time', 'longitude', 'latitude']
    meta_dict = dict()
    for k in metadata:
        meta_dict[k] = [df[k].loc[i].iloc[0] for i in ix]
    df = df.drop(metadata, axis=1)
    data_dict = {k:np.nan*np.ones((number_profiles,max_levels)) for k in df.columns}

    # put argo data into "square" arrays
    for nprof in ix:
        profile = df.loc[nprof]
        current_depth_levels = profile.shape[0]
        for k in profile.columns:
            data_dict[k][nprof-1,:current_depth_levels] = profile[k]
    

    # create xarray dataset
    ds = xr.Dataset(
        data_vars = {k:(['n_prof', 'n_level'], v) for k,v in data_dict.items()},
        coords = {k:(['n_prof'], v) for k,v in meta_dict.items()}
    )

    return ds



## This cell for Data Visualization

In [76]:
ds = dataframe_to_xarray(df)
ds

<xarray.Dataset>
Dimensions:       (n_prof: 13, n_level: 557)
Coordinates:
    wmo           (n_prof) int64 4903349 4903349 4903351 ... 4903346 5906438
    cycle_number  (n_prof) int64 0 1 0 0 0 0 0 1 0 1 0 1 1
    time          (n_prof) <U24 '2021-04-24T08:41:23.000Z' ... '2021-05-10T18...
    longitude     (n_prof) float64 -69.7 -69.72 -69.52 ... -65.6 -65.63 -66.82
    latitude      (n_prof) float64 14.15 14.17 15.2 17.08 ... 26.82 26.83 34.85
Dimensions without coordinates: n_prof, n_level
Data variables:
    pres          (n_prof, n_level) float64 1.04 2.04 3.0 ... 1.799e+03 1.9e+03
    psal          (n_prof, n_level) float64 35.91 35.92 35.92 ... 35.01 35.0
    temp          (n_prof, n_level) float64 27.28 27.28 27.28 ... 4.356 4.178

## This cell for Data Interpolation/Comparison